In [ ]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [ ]:
rotten = pd.read_csv("../INFO_PELIS/lista_pelis_2014_2020.csv", index_col = 0)

In [ ]:
rotten.tail()

In [ ]:
lista_titulos= rotten["Titulo"].to_list()

In [ ]:
lista_años= rotten["Año de estreno"].to_list()

In [ ]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})

In [ ]:
## Extraccion datos Rotten Tomatoes

url= "https://www.google.es/"


options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()
sleep(3)

puntuacion_tomatoes = {}
contador = 0
maximo = 5

for peli in list_busqueda_rotten[contador:maximo]:
    for titulo,año in peli.items():
         
        busqueda = f"TOMATOMETER title: {titulo} - year: {año}"
        # print(busqueda)
        
        try:
                       
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            sleep(random.randint(4,8))

            try:
                puntuacion_tomatoes[titulo]=driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text
                                
            except:
                puntuacion_completa= driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes[titulo]=puntuacion.group()
        
        except:
            puntuacion_tomatoes[titulo]="No existe puntuacion"



df_tomatoes = pd.Series(puntuacion_tomatoes,name="puntuacion")
df_tomatoes.index.name="titulo"
df_tomatoes.reset_index()
df_tomatoes.to_csv(f"{contador}_{maximo}_puntuacion_tomates_fedra.csv")